In [1]:
from rf_raster_vision_plugin.evaluate.config import VisionObjectDetectionEvaluatorConfigBuilder
from rf_raster_vision_plugin.raster_source.config import RfRasterSourceConfigBuilder
from rf_raster_vision_plugin.label_source.config import RfLabelSourceConfigBuilder
from rf_raster_vision_plugin.label_store.config import RfLabelStoreConfigBuilder
import rf_raster_vision_plugin.http.raster_foundry as rf
from rf_raster_vision_plugin.http import vision
import rastervision as rv
from rastervision.task.task_config import TaskConfig
from matplotlib import RcParams
import matplotlib.pyplot as plt
import numpy as np
import requests
from rastervision.core import Box
from rastervision.data.dataset_config import DatasetConfig
from rastervision.data.scene_config import SceneConfig
from rastervision.experiment.experiment_config import ExperimentConfig
from rastervision.rv_config import RVConfig
from uuid import uuid4
%matplotlib inline

# Setup objects in vision API

We'll need:

- a project
- an experiment

We're ignoring scenes for now, since there
[isn't an obvious way](https://github.com/azavea/raster-vision/blob/master/rastervision/evaluation/classification_evaluator.py#L23-L43)
to hook into per-scene evaluation.

In [2]:
rf_project_id = '71820687-526e-4203-8aab-3bf1acd5cc70' # a public project
rf_project_layer_id = '3cec2bbe-bb42-4df8-9478-bc8e57ab011e' # that project's default layer
rf_host = 'app.staging.rasterfoundry.com'
refresh_token = '9f8wgc5x2LH8o4j2LiB_MvxE-Yp4LmjwqaL35Vi9uDlYH'

vision_host = 'prediction.staging.rasterfoundry.com'

token = 'Bearer ' + rf.get_api_token(refresh_token, rf_host)
vision_project = vision.create_project(token, vision_host, 'object detection example')
vision_experiment = vision.create_experiment(
    token,
    vision_host,
    'object detection example',
    vision_project['id'],
    'super advanced ml model',
    'complicated and good',
    'object detection'
)


In [3]:
rs_config = RfRasterSourceConfigBuilder() \
    .with_project_id(rf_project_id) \
    .with_project_layer_id(rf_project_layer_id) \
    .with_refresh_token(refresh_token) \
    .with_channel_order([1, 2, 3]) \
    .with_num_channels(3) \
    .build()
raster_source = rs_config.create_source('/tmp')

label_source_config = RfLabelSourceConfigBuilder() \
    .with_annotation_group('59388cf6-5105-467c-a8f3-f098055be8f0') \
    .with_project_id(rf_project_id) \
    .with_project_layer_id(rf_project_layer_id) \
    .with_refresh_token(refresh_token) \
    .with_crs_transformer(raster_source.get_crs_transformer()) \
    .build()
label_source = label_source_config.create_source()

label_store_config = RfLabelStoreConfigBuilder() \
    .with_annotation_group('59388cf6-5105-467c-a8f3-f098055be8f0') \
    .with_project_id(rf_project_id) \
    .with_project_layer_id(rf_project_layer_id) \
    .with_refresh_token(refresh_token) \
    .with_crs_transformer(raster_source.get_crs_transformer()) \
    .with_class_map({v: k for k, v in label_source._class_map.items()}) \
    .build()



# then use the scene_config to make a dataset
task_config = TaskConfig.builder(rv.OBJECT_DETECTION) \
    .with_chip_size(200) \
    .with_classes({
        '870e82d4-0063-44f4-8a14-950266b23619': (1, 'red')
    }).build()

# use all the above to make a scene_config
scene_config = SceneConfig.builder() \
    .with_task(task_config) \
    .with_raster_source(rs_config) \
    .with_label_source(label_source_config) \
    .with_label_store(label_store_config) \
    .with_id('example scene') \
    .build()

# dataset config

dataset_config = DatasetConfig.builder() \
    .with_train_scenes([scene_config]) \
    .with_validation_scenes([scene_config]) \
    .with_test_scenes([scene_config]) \
    .build()

# then create an evaluator
evaluator = VisionObjectDetectionEvaluatorConfigBuilder() \
    .with_project_id(vision_project['id']) \
    .with_experiment_id(vision_experiment['id']) \
    .with_refresh_token(refresh_token) \
    .with_class_map(label_source._class_map) \
    .with_output_uri('/tmp') \
    .build()


backend_config = rv.BackendConfig.builder(rv.TF_OBJECT_DETECTION) \
    .with_task(task_config) \
    .with_debug(True) \
    .with_batch_size(16) \
    .with_num_steps(100000) \
    .with_model_defaults(rv.SSD_MOBILENET_V1_COCO) \
    .build()

# then create an experiment config that uses the evaluator and the task and the dataset
experiment_config = ExperimentConfig.builder() \
    .with_id('testexperiment') \
    .with_root_uri('/tmp') \
    .with_task(task_config) \
    .with_backend(backend_config) \
    .with_dataset(dataset_config) \
    .build()

2019-06-21 20:13:16:rastervision.utils.files: INFO - Downloading s3://raster-prediction-api-dev-andrew/clipped-cog-damage.tif to /tmp/tmpurjomyqn/s3/raster-prediction-api-dev-andrew/clipped-cog-damage.tif


{('LABEL_SOURCE', 'RF_ANNOTATION_GROUP_LABEL_SOURCE'): <class 'rf_raster_vision_plugin.label_source.config.RfLabelSourceConfigBuilder'>, ('RASTER_SOURCE', 'RF_LAYER_RASTER_SOURCE'): <class 'rf_raster_vision_plugin.raster_source.config.RfRasterSourceConfigBuilder'>, ('LABEL_STORE', 'RF_ANNOTATION_GROUP_LABEL_STORE'): <class 'rf_raster_vision_plugin.label_store.config.RfLabelStoreConfigBuilder'>, ('EVALUATOR', 'RF_RV_OBJECT_DETECTION_EVALUATOR'): <class 'rf_raster_vision_plugin.evaluate.config.VisionObjectDetectionEvaluatorConfigBuilder'>}


In [4]:
exp = experiment_config.fully_resolve()